# Interacting with the exposure log from a notebook

## Introduction

The exposurelog service exposes an endpoint that, depending on how it is called, can perform various actions on the exposure log stored in the database.
The intent of this notebook is to show examples of how one can wrap the underlying calls to make the interaction more like other client libraries.
The operations that can be done with the service endpoint are to:
* Get messages -- by default this is all the messages marked `is_valid`, but can be configured to return all messages
* Find a message by ID -- return a message as a dictionary for a particular message id if it exists
* Add a message -- Inserts a new message in the database.  By default, the observation ID must already exist in the repository, but if `is_new` is set to `True` the service will not do the check assuming that observation ID will show up in the repository eventually.
* Edit a message -- Edit an existing message.  In reality, this creates a new message in the database and marks the old one with `is_valid=False` and associating the new message with the old message ID as its parent.
* Delete a message -- This removes a message from the exposurelog, by marking the record associated with the message ID as invalid.
* Search for messages -- There are many ways for searching the messages and constraints can be stacked togehter.

## Import some helper functions

In principle there can be lots of instances of the exposure log service.
The `ENDPOINT` variable configures which particular instance you wish to query.

In [ ]:
from helpers import (ENDPOINT, add_message, get_messages, get_message_by_id, edit_message,
                     delete_message, MessageSearcher, ExposureSearcher)

## Let's try things out

First, list all valid messages.

By default this will return a `pandas.DataFrame`, but can be configured to return a list of dictionaries instead.
Only valid messages are returned unless `all=True` in which case both valid and invalid messages are returned.
By default the last 50 messages are returned.

In [ ]:
valid_messages = get_messages()
valid_messages.sort_values(by=['date_added'], ascending=False)  # Sort messages by when they are added with newer ones on top

Let's try adding a message.
We'll just give the minimum information.
Remember we will have to specify `is_new` since we don't have a valid observation id hanging around right now.
A copy of the message as it was ingested is returned.

If a `user_id` is not specified, it will try to use the username in this container.
If a `user_agent` is not specified, a default indicating that the message is coming from a notebook running in nublado.

> Note that currently the service does not provide validation of instrument names, so we will have to be fairly rigorous about our conventions for names of the various instruments

In [ ]:
message = add_message('Testing Obs ID', 'AuxTel', 'This is the message text used by the demo notebook', is_new=True)
message

Now we can get the message back that we put in.

Note this is a search by message id.
We will show how to search by observation_id a little later in this notebook.

In [ ]:
new_message = get_message_by_id(message['id'])
new_message

Why not fix up the message a little.
Notice that the parent_id in the edited message points to the message we originally added.

None of the message fields are required, but specifying none of them will result in an exact copy of the message matching `message_id`.
By default this will raise an exception if an invalid message is being edited, but this can be overridden.

In [ ]:
edit_message = edit_message(message['id'], message_text='An example of changing the message text after the fact')
edit_message

We can see both the parent message and the edited one by looking at the list of all messages.
Notice the original message still exists, but is now marked invalid.

In [ ]:
all_messages = get_messages(all=True)
all_messages.sort_values(by=['date_added'], ascending=False)

It turns out we don't want that message after all.
Remember we edited the message, so we need to use the id of the edited message, not the original we added.
Then list all messages to confirm it is now invalid.

In [ ]:
delete_message(edit_message['id'])

In [ ]:
all_messages = get_messages(all=True)
all_messages.sort_values(by=['date_added'], ascending=False)

So far, we have been dealing with ad hoc messages.
Let's try adding a message associated with an observation that exists in the registry.
First, get a list of the exposures by searching for them.
We will only specify the instrument which will get us the most recent 50 exposures.

In [ ]:
exps = ExposureSearcher(instrument='HSC').search()
exps

We'll just take the first observation ID and add a message associated with it

In [ ]:
obs_id = exps['obs_id'][0]
inst = exps['instrument'][0]
message = add_message(obs_id, inst, f'This is a test message associated with exposure {obs_id}')
message

Verify we have put a message in associated with that observation ID

In [ ]:
messages = MessageSearcher(obs_id=obs_id).search()
messages